In [6]:
import numpy as np
from collections import defaultdict

In [7]:
def keep_column():
    length = len(alignment[0])
    string_num = len(alignment)
    Keep_Column = np.ones(length).astype(int)
    Space_Count = np.zeros(length).astype(int)
    for i,string in  enumerate(alignment):
        for j,char in enumerate(string):
            if(char=="-"):
                Space_Count[j] +=1
    
    for i,count in enumerate(Space_Count):
        if(count/string_num > Theta):
            Keep_Column[i]=0
    
    return Keep_Column

In [15]:
def Initialize(length):
    Transition = defaultdict(dict)   ##hash of hash
    Emission   = defaultdict(dict)   ##hash of hash
    
    for symbol in alphabet:
        Emission['I0'][symbol]=0
    
    Transition['S']['I0'] = 0
    Transition['S']['M1'] = 0
    Transition['S']['D1'] = 0
    
    for i in range(1,length):
        
        for symbol in alphabet:
            Emission['M'+str(i)][symbol]=0
            Emission['I'+str(i)][symbol]=0
        
        Transition['I'+str(i-1)]['I'+str(i-1)] = 0
        Transition['I'+str(i-1)]['D'+str(i)] = 0
        Transition['I'+str(i-1)]['M'+str(i)] = 0
        
        Transition['D'+str(i)]['I'+str(i)] = 0
        Transition['D'+str(i)]['D'+str(i+1)] = 0
        Transition['D'+str(i)]['M'+str(i+1)] = 0
    
        Transition['M'+str(i)]['I'+str(i)] = 0
        Transition['M'+str(i)]['D'+str(i+1)] = 0
        Transition['M'+str(i)]['M'+str(i+1)] = 0
    
    
    for symbol in alphabet:
        Emission['I'+str(length)][symbol]=0
        Emission['M'+str(length)][symbol]=0
    
    
    Transition['I'+str(length-1)]['I'+str(length-1)] = 0
    Transition['I'+str(length-1)]['D'+str(length)] = 0
    Transition['I'+str(length-1)]['M'+str(length)] = 0
    
    Transition['I'+str(length)]['I'+str(length)] = 0
    Transition['I'+str(length)]['E']  = 0
    Transition['M'+str(length)]['I'+str(length)] = 0
    Transition['M'+str(length)]['E'] = 0
    Transition['D'+str(length)]['I'+str(length)] = 0
    Transition['D'+str(length)]['E'] = 0
    
    return Transition,Emission

In [2]:
def profile_hmm():
    string_num = len(alignment)
    ###determin which column should be removed
    Keep_Column = keep_column()   ###1 for keep, 0 for remove
    length = sum(Keep_Column)     ###length of match
    Transition,Emission = Initialize(length)
    for string in alignment:
        new_string = '' ####string excluding remove column
        for i,char in enumerate(string):
            if(Keep_Column[i]==1):
                new_string += char
        
        pre_state = 'S'        
        k = 0        
        for i,indicator in enumerate(Keep_Column):
            k += indicator
            if(indicator==1):
                if(new_string[k-1] == '-'):
                    state = 'D'+str(k)
                    Transition[pre_state][state] += 1
                    pre_state = state
                else:
                    state = 'M'+str(k)
                    Transition[pre_state][state] += 1
                    Emission[state][new_string[k-1]] += 1
                    pre_state = state
                    
            else:
                if(string[i] != '-'):
                    state = 'I'+str(k)
                    Transition[pre_state][state] += 1
                    Emission[state][string[i]] += 1
                    pre_state = state
            
        Transition[state]['E'] += 1
        
    return Transition,Emission,length

In [57]:
def Output(Transition,Emission,length):
    States = ['S','I0']
    for i in range(1,length+1):
        M = 'M'+str(i)
        States.append(M)
        D = 'D'+str(i)
        States.append(D)
        I = 'I'+str(i)
        States.append(I)
    States.append('E')
    
    ww = open("Profile.txt", "w")
    for state in States:
        ww.write('\t%s'%state)
    ww.write('\n')
    
    for state in States:
        trans_sum = sum(Transition[state].values())
        ww.write('%s'%state)
        if(trans_sum>0):
            sum_per = 0
            for state2 in States:
                if(state2 in Transition[state].keys()):
                    sum_per += Transition[state][state2]/trans_sum+sigma
                
            for state2 in States:
                if(state2 in Transition[state].keys()):
                    percent = (Transition[state][state2]/trans_sum+sigma)/sum_per
                    percent = ('%.3f' % percent)
                    percent = percent.rstrip('0').rstrip('.')
                    ww.write('\t%s'%percent)
                else:
                    ww.write('\t%d'%0)
        else:
            sum_per = 0
            for state2 in States:
                if(state2 in Transition[state].keys()):
                    sum_per += sigma
                
            for state2 in States: 
                if(state2 in Transition[state].keys()):
                    percent = sigma/sum_per
                    percent = ('%.3f' % percent)
                    percent = percent.rstrip('0').rstrip('.')
                    ww.write('\t%s'%percent)
                else:
                    ww.write('\t%d'%0)      
        ww.write('\n')
    
    
    ww.write('%s\n'%'--------')
    
    for symbol in alphabet:
        ww.write('\t%s'%symbol)
    ww.write('\n')
    
    for state in States:
        ww.write('%s'%state)
        if state in Emission.keys():
            emit_sum = sum(Emission[state].values())
            if(emit_sum==0):
                percent = 1/len(alphabet)
                percent = ('%.3f' % percent)
                percent = percent.rstrip('0').rstrip('.')
                for symbol in alphabet:    
                    ww.write('\t%s'%percent)
            else:
                sum_per = 1+len(alphabet)*sigma
                for symbol in alphabet:
                    percent = (Emission[state][symbol]/emit_sum + sigma)/sum_per
                    percent = ('%.3f' % percent)
                    percent = percent.rstrip('0').rstrip('.')
                    ww.write('\t%s'%percent)
        else:
            for symbol in alphabet:
                ww.write('\t%d'%0)
        ww.write('\n')  

In [58]:
Theta = 0.358
sigma = 0.01
alphabet = ['A', 'B', 'C', 'D', 'E']
alignment = ['A-A',
'ADA',
'ACA',
'A-C',
'-EA',
'D-A',]

In [59]:
Transition,Emission,length = profile_hmm()
Transition

defaultdict(dict,
            {'S': {'I0': 0, 'M1': 5, 'D1': 1},
             'I0': {'I0': 0, 'D1': 0, 'M1': 0},
             'D1': {'I1': 1, 'D2': 0, 'M2': 0},
             'M1': {'I1': 2, 'D2': 0, 'M2': 3},
             'I1': {'I1': 0, 'D2': 0, 'M2': 3},
             'I2': {'I2': 0, 'E': 0},
             'M2': {'I2': 0, 'E': 6},
             'D2': {'I2': 0, 'E': 0}})

In [60]:
Output(Transition,Emission,length)

In [61]:
Theta = 0.362
sigma = 0.01
alphabet = ['A', 'B', 'C', 'D', 'E']
alignment = ['--E',
'AE-',
'C-E',
'CE-',
'DDE']

In [62]:
Transition,Emission,length = profile_hmm()
Output(Transition,Emission,length)